# Experience Parallel Programming on Apache Spark

## Averages

### Set up the environment

In [1]:
from pyspark import SparkContext
sc = SparkContext()

### Generate the data

In [20]:
range(100)

range(0, 100)

In [53]:
list(range(100))

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99]

In [68]:
rdd = sc.parallelize([101] + list(range(100)) + [103, 104, 105, 106, 1000])

### Mean
Calculate the mean of the data generated.

In [69]:
sum = rdd.sum()
n = rdd.count()
mean = sum / n
print(mean)

61.028301886792455


### Median

The median is an **out-lier** resistant version of the mean.

In [71]:
sortedAndIndexed = rdd.sortBy(lambda x : x).zipWithIndex().map(lambda val: (val[1], val[0]))
n = sortedAndIndexed.count()
if(n % 2 == 1):
    index = (n - 1) / 2
    print(sortedAndIndexed.lookup(index)[0])
else:
    index1 = (n / 2) - 1
    index2 = n / 2
    value1 = sortedAndIndexed.lookup(index1)[0]
    value2 = sortedAndIndexed.lookup(index2)[0]
    print((value1 + value2) / 2)

52.5


## Standard Deviation

How wide the data is spread around the mean.

In [72]:
from math import sqrt
sqrt(rdd.map(lambda x : pow(x - mean, 2)).sum() / n)

96.5169315038959

## Skewness

How asymmetric data is spread about the mean.<br>
![](skewness.png)